In [9]:
%pylab inline

%pylab is deprecated, use %matplotlib inline and import the required libraries.
Populating the interactive namespace from numpy and matplotlib


In [10]:
import time
import numpy
from pandas import *
from tkinter import *

In [45]:
#----------------------------------- file parser ------------------------------------

"""
4 5     # number of rows and cols
A 4 0   # agent starting coordinates
W 1 0   # wumpus coordinates
G 1 1   # gold coordinates
P 0 3   # 1st pit coordinates
P 1 2   # 2nd pit coordinates
P 3 2   # 3rd pit coordinates
"""

class Maze_Pars:
    def __init__(self, maze_world):
        self.rowCol = []
        self.agent = []
        self.wumpus = []
        self.gold = []
        self.pits = [[]]
        file = open(maze_world, 'r')
        self.rowCol = file.readline()
        self.rowCol = self.rowCol.rstrip('\r\n')
        self.rowCol = self.rowCol.split(" ")

        self.agent = file.readline()
        self.agent = self.agent.rstrip('\r\n')
        self.agent = self.agent.split(" ")
        self.wumpus = file.readline()
        self.wumpus = self.wumpus.rstrip('\r\n')
        self.wumpus = self.wumpus.split(" ")
        self.gold = file.readline()
        self.gold = self.gold.rstrip('\r\n')
        self.gold = self.gold.split(" ")
        self.pits = []
        while True:
            pit = file.readline()
            if len(pit) == 0:
                break
            pit = pit.rstrip('\r\n')
            pit = pit.split(" ")
            self.pits.append(pit)

#----------------------------------- maze ------------------------------------

class Maze:
    def __init__(self):
        self.maze = [[]]
        self.rows = 0
        self.cols = 0
        self.agentRow = 0
        self.agentCol = 0
        self.caveEntranceRow = 0
        self.caveEntranceCol = 0

    def gen_maze(self, mazeWorld):
        Maze_pars = Maze_Pars(mazeWorld)
        self.rows = int(Maze_pars.rowCol[0])
        self.cols = int(Maze_pars.rowCol[1])
        self.maze = [[[] for i in range(self.cols)] for r in range(self.rows)]
        self.agentRow = int(Maze_pars.agent[1])
        self.agentCol = int(Maze_pars.agent[2])
        self.maze[self.agentRow][self.agentCol].append('A')
        self.maze[int(Maze_pars.wumpus[1])][int(Maze_pars.wumpus[2])].append(Maze_pars.wumpus[0])
        self.maze[int(Maze_pars.gold[1])][int(Maze_pars.gold[2])].append(Maze_pars.gold[0])
        for pit in Maze_pars.pits:
            self.maze[int(pit[1])][int(pit[2])].append(pit[0])
        self.display_signs()

    def display_signs(self):
        for i in range(self.rows):
            for r in range(self.cols):
                for c in range(len(self.maze[i][r])):
                    if self.maze[i][r][c] == 'W': # (W) Wumpus sign
                        try:
                            if i-1 >= 0:
                                if 'S' not in self.maze[i-1][r]:
                                    self.maze[i-1][r].append('S') # (S) Stench sign close to wumpus
                        except IndexError:
                            pass
                        try:
                            if r+1 < self.cols:
                                if 'S' not in self.maze[i][r+1]:
                                    self.maze[i][r+1].append('S')
                        except IndexError:
                            pass
                        try:
                            if i+1 < self.rows:
                                if 'S' not in self.maze[i+1][r]:
                                    self.maze[i+1][r].append('S')
                        except IndexError:
                            pass
                        try:
                            if r-1 >= 0:
                                if 'S' not in self.maze[i][r-1]:
                                    self.maze[i][r-1].append('S')
                        except IndexError:
                            pass

                    if self.maze[i][r][c] == 'P': # (P) Pits sign
                        try:
                            if i-1 >= 0:
                                if 'B' not in self.maze[i-1][r]:
                                    self.maze[i-1][r].append('B') # (B) Breeze sign close to pits
                        except IndexError:
                            pass
                        try:
                            if r+1 < self.cols:
                                if 'B' not in self.maze[i][r+1]:
                                    self.maze[i][r+1].append('B')
                        except IndexError:
                            pass
                        try:
                            if i+1 < self.rows:
                                if 'B' not in self.maze[i+1][r]:
                                    self.maze[i+1][r].append('B')
                        except IndexError:
                            pass
                        try:
                            if r-1 >= 0:
                                if 'B' not in self.maze[i][r-1]:
                                    self.maze[i][r-1].append('B')
                        except IndexError:
                            pass

#----------------------------------- agent ------------------------------------

"""
Legend:
. = visited tile
A = agent
G = gold
W = wumpus
S = stench
w = potential wumpus
nw = no wumpus
P = pit
B = breeze
p = potential pit
np = no pit
"""
class Agent:
    def __init__(self, maze, display_maze):
        self.maze = maze
        self.mazeKnowledge = [[[] for i in range(self.maze.cols)] for r in range(self.maze.rows)]
        self.mazeKnowledge[self.maze.agentRow][self.maze.agentCol].append('A')
        self.stenches = 0
        self.pathOutCave = [[self.maze.agentRow, self.maze.agentCol]]
        self.markTileVisited()
        self.maze.caveEntranceRow = self.maze.agentRow
        self.maze.caveEntranceCol = self.maze.agentCol
        self.mindedGold = False
        self.collectedGold = False
        self.exit = False
        self.display_maze = display_maze
        self.repaintMaze()

    def repaintMaze(self):
        for i in range(self.maze.rows):
            for r in range(self.maze.cols):
                textUpdate = []
                if 'A' in self.mazeKnowledge[i][r]:
                    textUpdate.append('A')
                if 'w' in self.mazeKnowledge[i][r]:
                    textUpdate.append('W')
                if 'p' in self.mazeKnowledge[i][r]:
                    textUpdate.append('P')
                if 'B' in self.mazeKnowledge[i][r]:
                    textUpdate.append('B')
                if 'S' in self.mazeKnowledge[i][r]:
                    textUpdate.append('S')
                if 'G' in self.mazeKnowledge[i][r]:
                    textUpdate.append('G')
                strUpdated=""
                self.display_maze[i][r].text_changed(strUpdated.join(textUpdate))
                if '.' in self.mazeKnowledge[i][r]:
                    self.display_maze[i][r].legend.config(bg="gray40")
                self.display_maze[i][r].legend.update()

    def move_back_tile(self):
        if self.maze.agentRow-1 == self.pathOutCave[-1][0]:
            self.move('u')
        if self.maze.agentRow+1 == self.pathOutCave[-1][0]:
            self.move('d')
        if self.maze.agentCol+1 ==  self.pathOutCave[-1][1]:
            self.move('r')
        if self.maze.agentCol-1 ==  self.pathOutCave[-1][1]:
            self.move('l')
        del self.pathOutCave[-1]

    def exit_cave(self):
        for tile in reversed(self.pathOutCave):
            if self.maze.agentRow-1 == tile[0]:
                self.move('u')
            if self.maze.agentRow+1 == tile[0]:
                self.move('d')
            if self.maze.agentCol+1 == tile[1]:
                self.move('r')
            if self.maze.agentCol-1 == tile[1]:
                self.move('l')
            if self.maze.caveEntranceRow == self.maze.agentRow:
                if self.maze.caveEntranceCol == self.maze.agentCol:
                    if self.mindedGold == True:
                        self.exit = True
                        break

    def explore_cave_for_gold(self):
        lastStep = ''
        ifMoved = False
        while self.mindedGold == False:
            if self.mindedGold == True:
                break
            try:
                if  '.' not in self.mazeKnowledge[self.maze.agentRow-1][self.maze.agentCol] and self.agentSafeStep(self.maze.agentRow-1, self.maze.agentCol):
                    if ifMoved == False:
                        if self.move('u'):
                            ifMoved = True
            except IndexError:
                pass
            try:
                if '.' not in self.mazeKnowledge[self.maze.agentRow][self.maze.agentCol+1] and self.agentSafeStep(self.maze.agentRow, self.maze.agentCol+1):
                    if ifMoved == False:
                        if self.move('r'):
                            ifMoved = True
            except IndexError:
                pass
            try:
                if '.' not in self.mazeKnowledge[self.maze.agentRow+1][self.maze.agentCol] and self.agentSafeStep(self.maze.agentRow+1, self.maze.agentCol):
                    if ifMoved == False:
                        if self.move('d'):
                            ifMoved = True
            except IndexError:
                pass
            try:
                if '.' not in self.mazeKnowledge[self.maze.agentRow][self.maze.agentCol-1] and self.agentSafeStep(self.maze.agentRow, self.maze.agentCol-1):
                    if ifMoved == False:
                        if self.move('l'):
                            ifMoved = True
            except IndexError:
                pass
            if ifMoved == False:
                self.move_back_tile()
            ifMoved = False

    def move(self, direction):
        self.repaintMaze()
        if self.mindedGold == True and self.collectedGold == False:
            self.collectedGold == True
            if 'G' in self.mazeKnowledge[self.maze.agentRow][self.maze.agentCol]:
                self.mazeKnowledge[self.maze.agentRow][self.maze.agentCol].remove('G')
        stepSucces = False
        if direction == 'u':
            if self.agentSafeStep(self.maze.agentRow-1, self.maze.agentCol):
                stepSucces = self.stepUp()
        if direction == 'r':
            if self.agentSafeStep(self.maze.agentRow, self.maze.agentCol+1):
                stepSucces = self.stepRight()
        if direction == 'd':
            if self.agentSafeStep(self.maze.agentRow+1, self.maze.agentCol):
                stepSucces = self.stepDown()
        if direction == 'l':
            if self.agentSafeStep(self.maze.agentRow, self.maze.agentCol-1):
                stepSucces = self.stepLeft()
        if stepSucces:
            self.signsKnowledge()
            self.markTileVisited()
            self.estimateWumpus()
            self.estimatePits()
            self.cleanEstimation()
            self.reassureWumpusKnowledge()
            if 'G' in self.mazeKnowledge[self.maze.agentRow][self.maze.agentCol]:
                self.mindedGold = True
            if self.mindedGold == False:
                self.pathOutCave.append([self.maze.agentRow, self.maze.agentCol])
            time.sleep(1.5)
        return stepSucces

    def signsKnowledge(self):
        if 'B' in self.maze.maze[self.maze.agentRow][self.maze.agentCol]:
            if 'B' not in self.mazeKnowledge[self.maze.agentRow][self.maze.agentCol]:
                self.mazeKnowledge[self.maze.agentRow][self.maze.agentCol].append('B')
        if 'S' in self.maze.maze[self.maze.agentRow][self.maze.agentCol]:
            if 'S' not in self.mazeKnowledge[self.maze.agentRow][self.maze.agentCol]:
                self.mazeKnowledge[self.maze.agentRow][self.maze.agentCol].append('S')
        if 'G' in self.maze.maze[self.maze.agentRow][self.maze.agentCol]:
            if 'G' not in self.mazeKnowledge[self.maze.agentRow][self.maze.agentCol]:
                self.mazeKnowledge[self.maze.agentRow][self.maze.agentCol].append('G')
        if 'P' in self.maze.maze[self.maze.agentRow][self.maze.agentCol]:
            if 'P' not in self.mazeKnowledge[self.maze.agentRow][self.maze.agentCol]:
                self.mazeKnowledge[self.maze.agentRow][self.maze.agentCol].append('P')
        if 'W' in self.maze.maze[self.maze.agentRow][self.maze.agentCol]:
            if 'W' not in self.mazeKnowledge[self.maze.agentRow][self.maze.agentCol]:
                self.mazeKnowledge[self.maze.agentRow][self.maze.agentCol].append('W')

    def estimatePits(self):
        try:
            if 'B' in self.maze.maze[self.maze.agentRow][self.maze.agentCol]:
                if self.maze.agentRow-1 >= 0:
                    if '.' not in self.maze.maze[self.maze.agentRow-1][self.maze.agentCol]:
                        if 'p' not in self.mazeKnowledge[self.maze.agentRow-1][self.maze.agentCol]:
                            self.mazeKnowledge[self.maze.agentRow-1][self.maze.agentCol].append('p')
        except IndexError:
            pass
        try:
            if 'B' in self.maze.maze[self.maze.agentRow][self.maze.agentCol]:
                if self.maze.agentCol+1 < self.maze.cols:
                    if '.' not in self.maze.maze[self.maze.agentRow][self.maze.agentCol+1]:
                        if 'p' not in self.mazeKnowledge[self.maze.agentRow][self.maze.agentCol+1]:
                            self.mazeKnowledge[self.maze.agentRow][self.maze.agentCol+1].append('p')
        except IndexError:
            pass
        try:
            if 'B' in self.maze.maze[self.maze.agentRow][self.maze.agentCol]:
                if self.maze.agentRow+1 < self.maze.rows:
                    if '.' not in self.maze.maze[self.maze.agentRow+1][self.maze.agentCol]:
                        if 'p' not in self.mazeKnowledge[self.maze.agentRow+1][self.maze.agentCol]:
                            self.mazeKnowledge[self.maze.agentRow+1][self.maze.agentCol].append('p')
        except IndexError:
            pass
        try:
            if 'B' in self.maze.maze[self.maze.agentRow][self.maze.agentCol]:
                if self.maze.agentCol-1 >= 0:
                    if '.' not in self.maze.maze[self.maze.agentRow][self.maze.agentCol-1]:
                        if 'p' not in self.mazeKnowledge[self.maze.agentRow][self.maze.agentCol-1]:
                            self.mazeKnowledge[self.maze.agentRow][self.maze.agentCol-1].append('p')
        except IndexError:
            pass

    def estimateWumpus(self):
        try:
            if 'S' in self.maze.maze[self.maze.agentRow][self.maze.agentCol]:
                if self.maze.agentRow-1 >= 0:
                    if '.' not in self.maze.maze[self.maze.agentRow-1][self.maze.agentCol]:
                        if 'w' not in self.mazeKnowledge[self.maze.agentRow-1][self.maze.agentCol]:
                            self.mazeKnowledge[self.maze.agentRow-1][self.maze.agentCol].append('w')
        except IndexError:
            pass
        try:
            if 'S' in self.maze.maze[self.maze.agentRow][self.maze.agentCol]:
                if self.maze.agentCol+1 < self.maze.cols:
                    if '.' not in self.maze.maze[self.maze.agentRow][self.maze.agentCol+1]:
                        if 'w' not in self.mazeKnowledge[self.maze.agentRow][self.maze.agentCol+1]:
                            self.mazeKnowledge[self.maze.agentRow][self.maze.agentCol+1].append('w')
        except IndexError:
            pass
        try:
            if 'S' in self.maze.maze[self.maze.agentRow][self.maze.agentCol]:
                if self.maze.agentRow+1 < self.maze.rows:
                    if '.' not in self.maze.maze[self.maze.agentRow+1][self.maze.agentCol]:
                        if 'w' not in self.mazeKnowledge[self.maze.agentRow+1][self.maze.agentCol]:
                            self.mazeKnowledge[self.maze.agentRow+1][self.maze.agentCol].append('w')
        except IndexError:
            pass
        try:
            if 'S' in self.maze.maze[self.maze.agentRow][self.maze.agentCol]:
                if self.maze.agentCol-1 >= 0:
                    if '.' not in self.maze.maze[self.maze.agentRow][self.maze.agentCol-1]:
                        if 'w' not in self.mazeKnowledge[self.maze.agentRow][self.maze.agentCol-1]:
                            self.mazeKnowledge[self.maze.agentRow][self.maze.agentCol-1].append('w')
        except IndexError:
            pass

    def cleanEstimation(self):
        self.stenches = 0
        for i in range(self.maze.rows):
            for r in range(self.maze.cols):
                if 'S' in self.mazeKnowledge[i][r]:
                    self.stenches += 1
                if 'w' in self.mazeKnowledge[i][r]:
                    try:
                        if i-1 >= 0:
                            if '.' in self.mazeKnowledge[i-1][r]:
                                if 'S' not in self.mazeKnowledge[i-1][r]:
                                    self.mazeKnowledge[i][r].remove('w')
                                    self.mazeKnowledge[i][r].append('nw')
                    except IndexError:
                        pass
                    try:
                        if r+1 < self.maze.cols:
                            if '.' in self.mazeKnowledge[i][r+1]:
                                if 'S' not in self.mazeKnowledge[i][r+1]:
                                    self.mazeKnowledge[i][r].remove('w')
                                    self.mazeKnowledge[i][r].append('nw')
                    except IndexError:
                        pass
                    try:
                        if i+1 < self.maze.rows:
                            if '.' in self.mazeKnowledge[i+1][r]:
                                if 'S' not in self.mazeKnowledge[i+1][r]:
                                    self.mazeKnowledge[i][r].remove('w')
                                    self.mazeKnowledge[i][r].append('nw')
                    except IndexError:
                        pass
                    try:
                        if r-1 >= 0:
                            if '.' in self.mazeKnowledge[i][r-1]:
                                if 'S' not in self.mazeKnowledge[i][r-1]:
                                    self.mazeKnowledge[i][r].remove('w')
                                    self.mazeKnowledge[i][r].append('nw')
                    except IndexError:
                        pass
                if 'p' in self.mazeKnowledge[i][r]:
                    try:
                        if i-1 >= 0:
                            if '.' in self.mazeKnowledge[i-1][r]:
                                if 'B' not in self.mazeKnowledge[i-1][r]:
                                    self.mazeKnowledge[i][r].remove('p')
                                    self.mazeKnowledge[i][r].append('np')
                    except IndexError:
                        pass
                    try:
                        if r+1 < self.maze.cols:
                            if '.' in self.mazeKnowledge[i][r+1]:
                                if 'B' not in self.mazeKnowledge[i][r+1]:
                                    self.mazeKnowledge[i][r].remove('p')
                                    self.mazeKnowledge[i][r].append('np')
                    except IndexError:
                        pass
                    try:
                        if i+1 < self.maze.rows:
                            if '.' in self.mazeKnowledge[i+1][r]:
                                if 'B' not in self.mazeKnowledge[i+1][r]:
                                    self.mazeKnowledge[i][r].remove('p')
                                    self.mazeKnowledge[i][r].append('np')
                    except IndexError:
                        pass
                    try:
                        if r-1 >= 0:
                            if '.' in self.mazeKnowledge[i][r-1]:
                                if 'B' not in self.mazeKnowledge[i][r-1]:
                                    self.mazeKnowledge[i][r].remove('p')
                                    self.mazeKnowledge[i][r].append('np')
                    except IndexError:
                        pass

    def reassureWumpusKnowledge(self):
        for i in range(self.maze.rows):
            for r in range(self.maze.cols):
                if 'w' in self.mazeKnowledge[i][r]:
                    stenchesCloseBy = 0
                    try:
                        if i-1 >= 0:
                            if 'S' in self.mazeKnowledge[i-1][r]:
                                stenchesCloseBy += 1
                    except IndexError:
                        pass
                    try:
                        if r+1 < self.maze.cols:
                            if 'S' in self.mazeKnowledge[i][r+1]:
                                stenchesCloseBy += 1
                    except IndexError:
                        pass
                    try:
                        if i+1 < self.maze.rows:
                            if 'S' in self.mazeKnowledge[i+1][r]:
                                stenchesCloseBy += 1
                    except IndexError:
                        pass
                    try:
                        if r-1 >= 0:
                            if 'S' in self.mazeKnowledge[i][r-1]:
                                stenchesCloseBy += 1
                    except IndexError:
                        pass
                    if stenchesCloseBy < self.stenches:
                        self.mazeKnowledge[i][r].remove('w')
                        self.mazeKnowledge[i][r].append('nw')

    def stepUp(self):
        try:
            if self.maze.agentRow-1 >= 0:
                self.clearAgent()
                self.maze.agentRow -= 1
                self.PutAgent()
                return True
            else:
                return False
        except IndexError:
            return False

    def stepRight(self):
        try:
            if self.maze.agentCol+1 < self.maze.cols:
                self.clearAgent()
                self.maze.agentCol += 1
                self.PutAgent()
                return True
            else:
                return False
        except IndexError:
            return False

    def stepDown(self):
        try:
            if self.maze.agentRow+1 < self.maze.rows:
                self.clearAgent()
                self.maze.agentRow += 1
                self.PutAgent()
                return True
            else:
                return False
        except IndexError:
            return False

    def stepLeft(self):
        try:
            if self.maze.agentCol-1 >= 0:
                self.clearAgent()
                self.maze.agentCol -= 1
                self.PutAgent()
                return True
            else:
                return False
        except IndexError:
            return False

    def clearAgent(self):
        self.maze.maze[self.maze.agentRow][self.maze.agentCol].remove('A')
        self.mazeKnowledge[self.maze.agentRow][self.maze.agentCol].remove('A')

    def PutAgent(self):
        self.maze.maze[self.maze.agentRow][self.maze.agentCol].append('A')
        self.mazeKnowledge[self.maze.agentRow][self.maze.agentCol].append('A')

    def markTileVisited(self):
        if '.' not in self.mazeKnowledge[self.maze.agentRow][self.maze.agentCol]:
            self.maze.maze[self.maze.agentRow][self.maze.agentCol].append('.')
            self.mazeKnowledge[self.maze.agentRow][self.maze.agentCol].append('.')

    def agentDead(self):
        if 'W' in self.maze.maze[self.maze.agentRow][self.maze.agentCol]:
            print("Agent dead, wumpus killed you!")
            return True
        elif 'P' in self.maze.maze[self.maze.agentRow][self.maze.agentCol]:
            print("Agent dead, fell into pit!")
            return True
        else:
            return False

    def agentSafeStep(self, row, col):
        try:
            if 'w' in self.mazeKnowledge[row][col]:
                return False
        except IndexError:
            pass
        try:
            if 'p' in self.mazeKnowledge[row][col]:
                return False
        except IndexError:
            pass
        try:
            if 'W' in self.mazeKnowledge[row][col]:
                return False
        except IndexError:
            pass
        try:
            if 'P' in self.mazeKnowledge[row][col]:
                return False
        except IndexError:
            pass
        return True


#----------------------------------- Grid legend ------------------------------------

class Display_Maze():
    def __init__(self, prime, i, r):
        self.text = StringVar()
        self.legend = Label(prime, textvariable = self.text, height = 5, width = 9, relief = GROOVE, bg = "black", fg = "white", font = "italic 12 bold")
        self.legend.grid(row = i, column = r, sticky = W, pady = 1)
        self.row = i
        self.col = r
    def text_changed(self, textUpdate):
        self.text.set(str(textUpdate))


#----------------------------------- GUI  ------------------------------------

def solve_wumpus_maze(prime, maze_world):
    maze = Maze()
    maze.gen_maze(maze_world)
    display_maze = [[Display_Maze(prime, i, r) for r in range(maze.cols)] for i in range(maze.rows)]
    agent = Agent(maze, display_maze)
    # Agent Solving
    while agent.exit == False:
        agent.explore_cave_for_gold()
        if agent.mindedGold == True:
            agent.exit_cave()
        break
    agent.repaintMaze()
    agent.mazeKnowledge[agent.maze.agentRow][agent.maze.agentCol].remove('A')
    time.sleep(1.5)
    agent.repaintMaze()

prime = Tk()
prime.title("Wumpus maze")
maze = Maze()
maze.gen_maze("maze_1.txt")
display_maze = [[Display_Maze(prime, i, r) for r in range(maze.cols)] for i in range(maze.rows)]
maze_1 = Button(prime, text="maze 1",  command= lambda: solve_wumpus_maze(prime, "maze_1.txt"), width=8, font = "Helvetica 14 bold", bg = "gray80", fg = "gray40", borderwidth=0, activeforeground="white", activebackground="gray40")
maze_1.grid(row = 0, column = len(display_maze[0]))

maze_2 = Button(prime, text="maze 2",  command= lambda: solve_wumpus_maze(prime, "maze_2.txt"), width=8, font = "Helvetica 14 bold", bg = "gray80", fg = "gray40", borderwidth=0, activeforeground="white", activebackground="gray40")
maze_2.grid(row = 1, column = len(display_maze[0]))

exit_button = Button(prime, text="Exit", command=prime.destroy, width=8, font = "Helvetica 14 bold", bg = "gray80", fg = "gray40", borderwidth=0, activeforeground="white", activebackground="gray40")
exit_button.grid(row=2, column = len(display_maze[0]))

mainloop()